In [1]:
import os
import json
import joblib
import time
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML
import torch

import nltk

# # Better pandas table
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [2]:
#connect to hugging face
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
install
!pip install -q ./transformers

fatal: destination path 'transformers' already exists and is not an empty directory.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.18.0.dev0 which is incompatible.


In [3]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [4]:
# Installing transformers
!pip install simpletransformers
#!pip install transformers
# Importing necessary libraries from simple transformers
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [5]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)

Using:  cuda


In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
# path_models = '/content/drive/MyDrive/w266/models/'
# path_outputs = '/content/drive/MyDrive/w266/outputs/'
# path_processed_data= '/content/drive/MyDrive/w266/processed_data/'

In [9]:
path_models = './models/cnn_2e/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'

In [10]:
data_name = 'cnn_test'
input_filename = data_name + '_prediction_ner_v3.json' 
#out_filename = data_name + '_summary_abs.json'

In [11]:

jsonFile = open(os.path.join(path_processed_data, input_filename), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

details = {
    'headlines' : details_dict['headlines'],
    'text' : details_dict['text'],
    'keywords' : details_dict['keywords'],
    'summary_art' : details_dict['summary_art'],
    'entities' : details_dict['entities'],
#    'text_ner' : details_dict['text_ner'],
    'sentence_1s' : details_dict['sentence_1s'],
    'sentence_3s' : details_dict['sentence_3s'],
    'summary_ext' : details_dict['summary_ext'],
    'summary_abs' : details_dict['summary_abs'],
    'summary_extabs' : details_dict['summary_extabs'],
    'summary_ner' : details_dict['summary_ner'],
    'summary_t5' : details_dict['summary_t5'],
    'predict_1s' : details_dict['predict_1s'],
    'predict_3s' : details_dict['predict_3s'],
    'predict_text' : details_dict['predict_text'],
    'predict_ext' : details_dict['predict_ext'],
    'predict_abs' : details_dict['predict_abs'],
    'predict_extabs' : details_dict['predict_extabs'],
#    'predict_ner' : details_dict['predict_ner'],
#    'predict_t5' : details_dict['predict_t5']
}

df_score = pd.DataFrame(details)

In [12]:
#print(len(details_dict['predict_ner']))

In [13]:
df_results = {k: details[k] for k in ('headlines', 'predict_1s', 'predict_3s', 'predict_text', 'predict_ext', 'predict_abs', 'predict_extabs')}
pd.DataFrame(df_results)

,headlines,predict_1s,predict_3s,predict_text,predict_ext,predict_abs,predict_extabs
0,Soldier slain London street machine gunner dru...,Story highlights Drummer Lee Riggers Rigby 25 ...,UK soldier slain cleaver became military recru...,UK soldier slain cleaver attack became militar...,UK soldier slain cleaver attack became militar...,UK soldier killed London attack became drummer,UK soldier killed soldier loved soccer loved life
1,New York probes report EMTs ignored woman died,New York Officials investigating reports two...,Fire dispatchers trained help pregnant worker ...,Fire dispatchers trained emergency medical car...,Fire dispatchers trained emergency medical car...,NYPD fire dispatchers charged death woman coll...,NYPD fire chief charged death 2 dispatchers
2,Erupt typical travel pattern,Editor note Share volcano photos CNN iReport .,Volcano summit The world hottest place ever see,Scientists hope Antarctica volcano discovery h...,Scientists hope Antarctica volcano discovery h...,The Antarctic volcano dreams . . ever.,The Antarctic volcano dreams . . ever.
3,Sentencing phase begin convicted Ohio serial k...,An Ohio jury Monday begin sentencing phase c...,Ohio jury begins sentencing phase convicted se...,Ohio jury begins sentencing phase convicted se...,Ohio serial killer Anthony Sowell goes sentenc...,ExCleveland man pleads guilty murder 11 women,Death row inmate gets life sentence Ohio killings
4,Girl loses 65 pounds fight childhood obesity,Update In July 2014 Breanna Bond 12 competed...,Weight loss inspires mom daughter fight fight ...,Weight loss battle childhood obesity,Weight loss helps kids fight diabetes,Teen loses 79 pounds loses 100 pounds,Weight loss inspires teen swimmer lose 100 pounds
...,...,...,...,...,...,...,...
9155,What evidence Syrian chemical weapons attack,outside experts help us establish whether thi...,Syrian opposition accuses regime using chemica...,Syria chemical weapons report No credible evid...,U. S. seeks probe alleged chemical weapons us...,Syria chemical weapons report No evidence chem...,Putin U. S. must step Iraq defense
9156,A town copes lives lost Texas blast,West Texas Since little boys growing West Te...,Texas brothers volunteer firefighters help sav...,Texas town mourns 9 firstresponders killed fire,9 firstresponders killed Texas blaze remembered,Firefighters killed blaze save lives,9 killed Colorado fire officials say
9157,Voluntary recall another compounding facility,Government Accountability Office released rep...,FDA investigating deadly compounding pharmacy ...,FDA orders new look compounding pharmacy safety,FDA orders new look compounding pharmacy safety,FDA probes possible link tainted drugs linked ...,FDA probes possible link tainted drugs linked ...
9158,Crowdsourced maps help mobile users compare ne...,Editor note Amy Gahran writes mobile tech CNN .,Mobile apps help kids get ahead mobile apps,Why mobile apps matter,Why mobile apps matter,Why I quit Facebook,Google launches mobile app search results


In [14]:
pip install Rouge

Note: you may need to restart the kernel to use updated packages.


In [15]:
from rouge import Rouge
rouge = Rouge()

In [16]:
score_1s = rouge.get_scores(details_dict['predict_1s'], details_dict['headlines'], avg=True)
df_results = {k: score_1s[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.272922,0.089370,0.241525
p,0.109200,0.030277,0.098448
f,0.142266,0.041741,0.126898


In [17]:
score_3s = rouge.get_scores(details_dict['predict_3s'], details_dict['headlines'], avg=True)
df_results = {k: score_3s[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.283993,0.110531,0.265490
p,0.282199,0.108220,0.263905
f,0.277124,0.106598,0.259101


In [18]:
score_text = rouge.get_scores(details_dict['predict_text'], details_dict['headlines'], avg=True)
df_results = {k: score_text[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.312894,0.119527,0.291087
p,0.306454,0.114797,0.284996
f,0.303171,0.114123,0.281948


In [19]:
score_ext = rouge.get_scores(details_dict['predict_ext'], details_dict['headlines'], avg=True)
df_results = {k: score_ext[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.302569,0.114378,0.281460
p,0.299093,0.111136,0.278101
f,0.294356,0.109771,0.273707


In [20]:
score_abs = rouge.get_scores(details_dict['predict_abs'], details_dict['headlines'], avg=True)
df_results = {k: score_abs[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.233984,0.079583,0.220967
p,0.242024,0.081581,0.228319
f,0.232929,0.078454,0.219844


In [21]:
score_extabs = rouge.get_scores(details_dict['predict_extabs'], details_dict['headlines'], avg=True)
df_results = {k: score_extabs[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.225712,0.075680,0.212256
p,0.234660,0.077775,0.220335
f,0.225142,0.074592,0.211520


In [22]:
# score_ner = rouge.get_scores(details_dict['predict_ner'], details_dict['headlines'], avg=True)
# df_results = {k: score_ner[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
# pd.DataFrame(df_results)

In [23]:
# score_t5 = rouge.get_scores(details_dict['predict_t5'], details_dict['headlines'], avg=True)
# df_results = {k: score_t5[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
# pd.DataFrame(df_results)

In [24]:
score_art = rouge.get_scores(details_dict['predict_art'], details_dict['headlines'], avg=True)
df_results = {k: score_art[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.343929,0.137817,0.320208
p,0.337781,0.132888,0.314184
f,0.333528,0.131681,0.310347


In [25]:
rouge_1 = {'baseline (1s)': {'r': score_1s['rouge-1']['r'], 'p':score_1s['rouge-1']['p'], 'f': score_1s['rouge-1']['f']}, 
        'baseline (3s)': {'r': score_3s['rouge-1']['r'], 'p':score_3s['rouge-1']['p'], 'f': score_3s['rouge-1']['f']}, 
           'text': {'r': score_text['rouge-1']['r'], 'p':score_text['rouge-1']['p'], 'f': score_text['rouge-1']['f']},
           'ext': {'r': score_ext['rouge-1']['r'], 'p':score_ext['rouge-1']['p'], 'f': score_ext['rouge-1']['f']},
           'abs': {'r': score_abs['rouge-1']['r'], 'p':score_abs['rouge-1']['p'], 'f': score_abs['rouge-1']['f']},
           'extabs': {'r': score_extabs['rouge-1']['r'], 'p':score_extabs['rouge-1']['p'], 'f': score_extabs['rouge-1']['f']},
           'artsum': {'r': score_art['rouge-1']['r'], 'p':score_art['rouge-1']['p'], 'f': score_art['rouge-1']['f']},
#           't5': {'r': score_t5['rouge-1']['r'], 'p':score_t5['rouge-1']['p'], 'f': score_t5['rouge-1']['f']},
}

In [26]:
df_results = {k: rouge_1[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'baseline (1s)', 'abs', 'extabs', )}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),baseline (1s),abs,extabs
r,0.343929,0.312894,0.302569,0.283993,0.272922,0.233984,0.225712
p,0.337781,0.306454,0.299093,0.282199,0.109200,0.242024,0.234660
f,0.333528,0.303171,0.294356,0.277124,0.142266,0.232929,0.225142


In [27]:
rouge_2 = {'baseline (1s)': {'r': score_1s['rouge-2']['r'], 'p':score_1s['rouge-2']['p'], 'f': score_1s['rouge-2']['f']}, 
        'baseline (3s)': {'r': score_3s['rouge-2']['r'], 'p':score_3s['rouge-2']['p'], 'f': score_3s['rouge-2']['f']}, 
           'text': {'r': score_text['rouge-2']['r'], 'p':score_text['rouge-2']['p'], 'f': score_text['rouge-2']['f']},
           'ext': {'r': score_ext['rouge-2']['r'], 'p':score_ext['rouge-2']['p'], 'f': score_ext['rouge-2']['f']},
           'abs': {'r': score_abs['rouge-2']['r'], 'p':score_abs['rouge-2']['p'], 'f': score_abs['rouge-2']['f']},
           'extabs': {'r': score_extabs['rouge-2']['r'], 'p':score_extabs['rouge-2']['p'], 'f': score_extabs['rouge-2']['f']},
           'artsum': {'r': score_art['rouge-2']['r'], 'p':score_art['rouge-2']['p'], 'f': score_art['rouge-2']['f']},
#           't5': {'r': score_t5['rouge-2']['r'], 'p':score_t5['rouge-2']['p'], 'f': score_t5['rouge-2']['f']},
}
df_results = {k: rouge_2[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'baseline (1s)', 'abs', 'extabs', )}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),baseline (1s),abs,extabs
r,0.137817,0.119527,0.114378,0.110531,0.089370,0.079583,0.075680
p,0.132888,0.114797,0.111136,0.108220,0.030277,0.081581,0.077775
f,0.131681,0.114123,0.109771,0.106598,0.041741,0.078454,0.074592


In [30]:
rouge_l = {'baseline (1s)': {'r': score_1s['rouge-l']['r'], 'p':score_1s['rouge-l']['p'], 'f': score_1s['rouge-l']['f']}, 
        'baseline (3s)': {'r': score_3s['rouge-l']['r'], 'p':score_3s['rouge-l']['p'], 'f': score_3s['rouge-l']['f']}, 
           'text': {'r': score_text['rouge-l']['r'], 'p':score_text['rouge-l']['p'], 'f': score_text['rouge-l']['f']},
           'ext': {'r': score_ext['rouge-l']['r'], 'p':score_ext['rouge-l']['p'], 'f': score_ext['rouge-l']['f']},
           'abs': {'r': score_abs['rouge-l']['r'], 'p':score_abs['rouge-l']['p'], 'f': score_abs['rouge-l']['f']},
           'extabs': {'r': score_extabs['rouge-l']['r'], 'p':score_extabs['rouge-l']['p'], 'f': score_extabs['rouge-l']['f']},
           'artsum': {'r': score_art['rouge-l']['r'], 'p':score_art['rouge-l']['p'], 'f': score_art['rouge-l']['f']},
#           't5': {'r': score_t5['rouge-l']['r'], 'p':score_t5['rouge-l']['p'], 'f': score_t5['rouge-l']['f']},
}
df_results = {k: rouge_l[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'baseline (1s)', 'abs', 'extabs', )}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),baseline (1s),abs,extabs
r,0.320208,0.291087,0.281460,0.265490,0.241525,0.220967,0.212256
p,0.314184,0.284996,0.278101,0.263905,0.098448,0.228319,0.220335
f,0.310347,0.281948,0.273707,0.259101,0.126898,0.219844,0.211520


In [29]:
# Evaluation Metrics - BLEU implementation
from nltk.translate.bleu_score import sentence_bleu


In [30]:
# Function to calculate the score
def calc_bleu(x, y):
    L = 0
    for i, j in zip(x, y):
      L += sentence_bleu([i], j, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=None,  auto_reweigh=False,)

    # Average BLEU score of whole corpuses
    return round(L/len(x), 4)



In [31]:
bleu_1s = calc_bleu(details_dict['headlines'], details_dict['predict_1s'])
print("BLEU score: {:.04f}".format(bleu_1s))

/home/ubuntu/anaconda3/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

BLEU score: 0.1328


In [32]:
bleu_3s = calc_bleu(details_dict['headlines'], details_dict['predict_3s'])
print("BLEU score: {:.04f}".format(bleu_3s))

BLEU score: 0.3128


In [33]:
bleu_text = calc_bleu(details_dict['headlines'], details_dict['predict_text'])
print("BLEU score: {:.04f}".format(bleu_text))

BLEU score: 0.3408


In [34]:
bleu_ext = calc_bleu(details_dict['headlines'], details_dict['predict_ext'])
print("BLEU score: {:.04f}".format(bleu_ext))

BLEU score: 0.3325


In [35]:
bleu_abs = calc_bleu(details_dict['headlines'], details_dict['predict_abs'])
print("BLEU score: {:.04f}".format(bleu_abs))

BLEU score: 0.2777


In [36]:
bleu_extabs = calc_bleu(details_dict['headlines'], details_dict['predict_extabs'])
print("BLEU score: {:.04f}".format(bleu_extabs))

BLEU score: 0.2700


In [37]:
# bleu_ner = calc_bleu(details_dict['headlines'], details_dict['predict_ner'])
# print("BLEU score: {:.04f}".format(bleu_ner))

In [38]:
# bleu_t5 = calc_bleu(details_dict['headlines'], details_dict['predict_t5'])
# print("BLEU score: {:.04f}".format(bleu_t5))

In [39]:
bleu_art = calc_bleu(details_dict['headlines'], details_dict['predict_art'])
print("BLEU score: {:.04f}".format(bleu_art))

BLEU score: 0.3691


In [40]:
bleu_score = {'baseline (1s)': {'bleu_score': bleu_1s}, 
                'baseline (3s)': {'bleu_score': bleu_3s}, 
    'text' : {'bleu_score': bleu_text}, 
    'ext' : {'bleu_score': bleu_ext},
    'abs' : {'bleu_score': bleu_abs},
    'extabs' : {'bleu_score': bleu_extabs},
    'artsum' : {'bleu_score': bleu_art},
#    't5' : {'bleu_score': bleu_t5}
}

In [41]:
df_results = {k: bleu_score[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'abs', 'extabs', 'baseline (1s)')}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),abs,extabs,baseline (1s)
bleu_score,0.3691,0.3408,0.3325,0.3128,0.2777,0.27,0.1328


In [42]:
pip install bert-score

Note: you may need to restart the kernel to use updated packages.


In [43]:
from bert_score import score

In [44]:
# Baseline
P, R, F1 = score(details_dict['predict_1s'], details_dict['headlines'], lang="en", verbose=True)
F1_1s = F1.mean().item()
P_1s = P.mean().item()
R_1s = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/274 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 57.26 seconds, 159.96 sentences/sec


In [45]:
# Ext
P, R, F1 = score(details_dict['predict_3s'], details_dict['headlines'], lang="en", verbose=True)
F1_3s = F1.mean().item()
P_3s = P.mean().item()
R_3s = R.mean().item()


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/281 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 37.69 seconds, 243.06 sentences/sec


In [46]:
# Abs
P, R, F1 = score(details_dict['predict_text'], details_dict['headlines'], lang="en", verbose=True)
F1_text = F1.mean().item()
P_text = P.mean().item()
R_text = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/284 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 38.65 seconds, 236.97 sentences/sec


In [47]:
# ExtAbs
P, R, F1 = score(details_dict['predict_ext'], details_dict['headlines'], lang="en", verbose=True)
F1_ext = F1.mean().item()
P_ext = P.mean().item()
R_ext = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/283 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 39.31 seconds, 233.02 sentences/sec


In [48]:
P, R, F1 = score(details_dict['predict_abs'], details_dict['headlines'], lang="en", verbose=True)
F1_abs = F1.mean().item()
P_abs = P.mean().item()
R_abs = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/283 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 39.40 seconds, 232.50 sentences/sec


In [49]:
P, R, F1 = score(details_dict['predict_extabs'], details_dict['headlines'], lang="en", verbose=True)
F1_extabs = F1.mean().item()
P_extabs = P.mean().item()
R_extabs = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/281 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 39.92 seconds, 229.47 sentences/sec


In [50]:
# P, R, F1 = score(details_dict['predict_ner'], details_dict['headlines'], lang="en", verbose=True)
# F1_ner = F1.mean().item()
# P_ner = P.mean().item()
# R_ner = R.mean().item()

In [51]:
# P, R, F1 = score(details_dict['predict_t5'], details_dict['headlines'], lang="en", verbose=True)
# F1_t5 = F1.mean().item()
# P_t5 = P.mean().item()
# R_t5 = R.mean().item()

In [52]:
P, R, F1 = score(details_dict['predict_art'], details_dict['headlines'], lang="en", verbose=True)
F1_art = F1.mean().item()
P_art = P.mean().item()
R_art = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/283 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/144 [00:00<?, ?it/s]

done in 41.52 seconds, 220.64 sentences/sec


In [53]:
# import matplotlib.pyplot as plt
# plt.hist(F1, bins=20)
# plt.show()
# from bert_score import plot_example

# p1 = P1[0]
# y = Y[0]
# plot_example(p1, y, lang="en")

In [54]:
# bert_score = {'baseline': {'P': P_base,
#   'R': R_base,
#   'F1': F1_base}, 
#    'ext' : {'P': P_ext,
#   'R': R_ext,
#   'F1': F1_ext}, 
#   'abs' : {'P': P_abs,
#   'R': R_abs,
#   'F1': F1_abs},
#   'extabs' : {'P': P_extabs,
#   'R': R_extabs,
#   'F1': F1_extabs},
# }

bert_score = {'P': {  'art': P_art,
                      'text': P_text,
                      'ext': P_ext,
  'baseline (3 sentences)': P_3s,

  'abs': P_abs,
  'extabs': P_extabs,
'baseline (1 sentence)': P_1s,}, 
   'R' : {  'art': R_art,
            'text': R_text,
            'ext': R_ext,
  'baseline (3 sentences)': R_3s,
  'abs': R_abs,
  'extabs': R_extabs,
   'baseline (1 sentence)': R_1s,}, 
  'F1' : {  'art': F1_art,
            'text': F1_text,
            'ext': F1_ext,
  'baseline (3 sentences)': F1_3s,
  'abs': F1_abs,
  'extabs': F1_extabs,
  'baseline (1 sentence)': F1_1s,},
}



In [55]:
df_results = {k: bert_score[k] for k in ('P', 'R', 'F1')}
pd.DataFrame(df_results)

,P,R,F1
art,0.885727,0.882444,0.883887
text,0.882811,0.879360,0.880891
ext,0.881873,0.877882,0.879682
baseline (3 sentences),0.879829,0.874897,0.877168
abs,0.875696,0.868051,0.871677
extabs,0.874679,0.866587,0.870435
baseline (1 sentence),0.819547,0.851331,0.834391


In [56]:
#!pip install --upgrade pip  # ensures that pip is current
#!git clone https://github.com/google-research/bleurt.git
#!cd bleurt
#!pip install .

In [57]:
# import unittest

# python -m unittest bleurt.score_test
# python -m unittest bleurt.score_not_eager_test
# python -m unittest bleurt.finetune_test
# python -m unittest bleurt.score_files_test

In [58]:
from functools import reduce
  
def average(lst):
    return reduce(lambda a, b: a + b, lst) / len(lst)

In [59]:
from bleurt import score

In [60]:
checkpoint = "bleurt/bleurt/test_checkpoint"
scorer = score.BleurtScorer(checkpoint)

INFO:tensorflow:Reading checkpoint bleurt/bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


In [61]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_1s']) 
bleurt_1s = average(scores)

In [62]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_3s']) 
bleurt_3s = average(scores)

In [63]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_text']) 
bleurt_text = average(scores)

In [64]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_ext']) 
bleurt_ext = average(scores)

In [65]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_abs']) 
bleurt_abs = average(scores)

In [66]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_extabs']) 
bleurt_extabs = average(scores)

In [67]:
# scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_ner']) 
# bleurt_ner = average(scores)

In [68]:
# scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_t5']) 
# bleurt_t5 = average(scores)

In [69]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_art']) 
bleurt_art = average(scores)

In [70]:
bleurt_score = {'baseline (1s)': {'bleurt_score': bleurt_1s}, 
                'baseline (3s)': {'bleurt_score': bleurt_3s}, 
    'text' : {'bleurt_score': bleurt_text}, 
    'ext' : {'bleurt_score': bleurt_ext},
    'abs' : {'bleurt_score': bleurt_abs},
    'extabs' : {'bleurt_score': bleurt_extabs},
#    'ner' : {'bleurt_score': bleurt_ner},
#    't5' : {'bleurt_score': bleurt_t5},
    'art' : {'bleurt_score': bleurt_art},                
}

# bleurt_score = {'bleurt_score': {'baseline': bleurt_baseline}, 
#    'bleurt_score' : {'ext': bleurt_ext}, 
#   'bleurt_score' : {'abs': bleurt_abs},
#   'bleurt_score' : {'extabs': bleurt_extabs}
# }

#bleurt_score = {'bleurt_score': {'baseline': bleurt_baseline, 'ext': bleurt_ext, 'abs': bleurt_abs, 'extabs': bleurt_extabs}}

In [71]:
#df_results = {k: bleurt_score[k] for k in ('bleurt_score')}
df_results = {k: bleurt_score[k] for k in ('art', 'text', 'ext', 'baseline (3s)', 'abs', 'extabs', 'baseline (1s)')}
pd.DataFrame(df_results)

,art,text,ext,baseline (3s),abs,extabs,baseline (1s)
bleurt_score,-0.563968,-0.615786,-0.633653,-0.670758,-0.7601,-0.778038,-0.811343


In [72]:
#score = nltk.translate.meteor_score.meteor_score(reference, model)
# print (nltk.translate.meteor_score.meteor_score(
#     ["this is an apple", "that is an apple"], "an apple on this tree"))
# print (nltk.translate.meteor_score.meteor_score(
#     ["this is an apple", "that is an apple"], "a red color fruit"))
# nltk.translate.meteor_score.meteor_score(P1, "a red color fruit")

In [73]:
# meteor_score_1s = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_1s']):
#     meteor_score_1s += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_1s = meteor_score_1s/len(details_dict['headlines'])
# meteor_score_1s

In [74]:
# meteor_score_3s = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_3s']):
#     meteor_score_3s += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_3s = meteor_score_3s/len(details_dict['headlines'])
# meteor_score_3s

In [75]:
# meteor_score_text = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_text']):
#     meteor_score_text += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_text = meteor_score_text/len(details_dict['headlines'])
# meteor_score_text

In [76]:
# meteor_score_ext = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_ext']):
#     meteor_score_ext += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_ext = meteor_score_ext/len(details_dict['headlines'])
# meteor_score_ext

In [77]:
# meteor_score_abs = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_abs']):
#     meteor_score_abs += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_abs = meteor_score_abs/len(details_dict['headlines'])
# meteor_score_abs

In [78]:
# meteor_score_extabs = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_extabs']):
#     meteor_score_extabs += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_extabs = meteor_score_extabs/len(details_dict['headlines'])
# meteor_score_extabs

In [79]:
# meteor_score_ner = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_ner']):
#     meteor_score_ner += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_ner = meteor_score_ner/len(details_dict['headlines'])
# meteor_score_ner

In [80]:
# meteor_score_t5 = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_t5']):
#     meteor_score_t5 += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_t5 = meteor_score_t5/len(details_dict['headlines'])
# meteor_score_t5

In [81]:
# meteor_score_art = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_art']):
#     meteor_score_art += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_art = meteor_score_art/len(details_dict['headlines'])
# meteor_score_art

In [82]:
# meteor_score = {'baseline (1s)': {'meteor_score': meteor_score_1s}, 
#                 'baseline (3s)': {'meteor_score': meteor_score_3s}, 
#     'text' : {'meteor_score': meteor_score_text}, 
#     'ext' : {'meteor_score': meteor_score_ext},
#     'abs' : {'meteor_score': meteor_score_abs},
#     'extabs' : {'meteor_score': meteor_score_extabs},
#     'ner' : {'meteor_score': meteor_score_ner},
#     't5' : {'meteor_score': meteor_score_t5},
#     'art' : {'meteor_score': meteor_score_art}
# }
# df_results = {k: meteor_score[k] for k in ('baseline (1s)', 'baseline (3s)', 'text', 'ext', 'abs', 'extabs', 'ner', 't5', 'art')}
# pd.DataFrame(df_results)

In [83]:
len(details_dict['headlines'])

9160